# CNN + Data Transfer Learning 

## Equipo 4:


*   Karla Andrea Palma Villanueva (A01754270)
*   Viviana Alanis Fraige (A01236316)
* David Fernando Armendariz Torres (A01570813)
* Alan Alberto Mota Yescas (A01753924)
* Adrián Chávez Morales (A01568679)
* Jose Manuel Armendáriz Mena (A01197583)

### Introducción

El objetivo de este notebook es desarrollar un modelo de Transfer Learning utilizando MobileNetV2 para la clasificación de imágenes del dataset Stanford Dogs, que contiene 120 clases de razas caninas. A lo largo del proyecto, se busca aplicar técnicas avanzadas de aprendizaje profundo para construir un modelo eficiente que pueda generalizar correctamente en datos nunca antes vistos.

El enfoque principal es aprovechar modelos preentrenados en ImageNet, como MobileNetV2, para reducir los tiempos de entrenamiento y mejorar la precisión, dada la complejidad de las tareas de visión por computadora. Además, se exploran varias estrategias, como Fine-Tuning, Data Augmentation, y ajustes de hiperparámetros para evitar el sobreajuste y mejorar el rendimiento del modelo.

### Exploración, explicación y limpieza de datos

#### Origen y Contexto del Dataset Stanford dogs


El dataset utilizado para este proyecto es Stanford Dogs, el cual fue desarrollado por investigadores de la Universidad de Stanford para abordar problemas avanzados de clasificación de imágenes y reconocimiento de objetos específicos. Este dataset se ha utilizado ampliamente en tareas de visión por computadora debido a su alta variabilidad y el nivel de detalle requerido para distinguir entre las diferentes razas de perros.

El dataset contiene 20,580 imágenes de perros de 120 razas distintas, siguiendo las categorías del American Kennel Club (AKC). Cada clase corresponde a una raza, lo que plantea un desafío significativo para los modelos de aprendizaje profundo debido a:
- La alta similitud entre algunas razas, como las variantes de retrievers y spaniels.
- La diversidad en las condiciones de captura de las imágenes, incluyendo variaciones en iluminación, fondo y poses de los perros.

#### Obtención del Dataset
El dataset está disponible públicamente en la plataforma de Stanford y puede descargarse desde:
- [Stanford Dogs Dataset](http://vision.stanford.edu/aditya86/ImageNetDogs/)
- Tamaño: Aproximadamente 750 MB
- Formato: Las imágenes están organizadas en carpetas, donde cada carpeta representa una raza específica de perro.

Este dataset es particularmente útil para evaluar la capacidad de generalización de modelos preentrenados debido a su granularidad y la necesidad de que los algoritmos diferencien sutiles características entre las razas.


In [16]:
#Importación de librerías
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping



In [2]:
# Cargar el dataset Stanford Dogs
dataset_name = 'stanford_dogs'
(raw_train, raw_test), info = tfds.load(
    dataset_name,
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,  
    with_info=True  
)

#información del dataset
print(info)

for image, label in raw_train.take(1):
    print(f"Label: {label}")
    tf.keras.preprocessing.image.array_to_img(image).show()

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

...

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Alan\tensorflow_datasets\stanford_dogs\incomplete.11PDXG_0.2.0\stanford_dogs-train.tfrecord…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Alan\tensorflow_datasets\stanford_dogs\incomplete.11PDXG_0.2.0\stanford_dogs-test.tfrecord*…

Dataset stanford_dogs downloaded and prepared to C:\Users\Alan\tensorflow_datasets\stanford_dogs\0.2.0. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='stanford_dogs',
    full_name='stanford_dogs/0.2.0',
    description="""
    The Stanford Dogs dataset contains images of 120 breeds of dogs from around the
    world. This dataset has been built using images and annotation from ImageNet for
    the task of fine-grained image categorization. There are 20,580 images, out of
    which 12,000 are used for training and 8580 for testing. Class labels and
    bounding box annotations are provided for all the 12,000 images.
    """,
    homepage='http://vision.stanford.edu/aditya86/ImageNetDogs/main.html',
    data_dir='C:\\Users\\Alan\\tensorflow_datasets\\stanford_dogs\\0.2.0',
    file_format=tfrecord,
    download_size=778.12 MiB,
    dataset_size=744.72 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filen

### Análisis del Dataset: Número de Columnas, Instancias, y Tipos de Datos

El dataset Stanford Dogs consta de 20,580 imágenes organizadas en 120 clases correspondientes a diferentes razas de perros. Este conjunto de datos fue diseñado específicamente para abordar el problema de la clasificación fina de imágenes, un reto en visión por computadora donde las clases tienen características muy sutiles entre sí. Las imágenes y anotaciones provienen de ImageNet, lo que garantiza la calidad y diversidad del dataset. 

Cada entrada en el dataset incluye una imagen en formato RGB, con tres canales de color (rojo, verde y azul), lo que permite trabajar con imágenes a color. Estas imágenes pueden tener diferentes resoluciones, por lo que será necesario redimensionarlas para el modelo de aprendizaje profundo. Además, se proporciona el nombre del archivo de cada imagen en formato de texto (string) y un etiquetado numérico (label) en formato entero (int64), que representa la raza a la que pertenece el perro en la imagen. En total, se identifican 120 clases de razas distintas.

El dataset también incluye coordenadas de bounding boxes en algunas imágenes, las cuales se almacenan en un formato de secuencia (float32). Estos bounding boxes delimitan las áreas de interés en la imagen donde se encuentran los perros, lo que puede ser útil en tareas de detección de objetos.

La división del dataset se realiza en dos subconjuntos: 12,000 imágenes destinadas para el entrenamiento y 8,580 imágenes para la prueba. Esta partición sigue la estructura estándar para entrenar y evaluar modelos de clasificación, lo que permite medir la capacidad del modelo para generalizar en datos no vistos previamente. Las imágenes se almacenan en el formato TFRecord, una estructura binaria eficiente que facilita la manipulación de grandes volúmenes de datos durante el entrenamiento.

El tamaño total del dataset es de 744.72 MB después de la extracción, mientras que la descarga inicial ocupa 778.12 MB. Gracias a su configuración supervisada, cada entrada del dataset asocia una imagen con su etiqueta correspondiente, lo que facilita la aplicación de técnicas de aprendizaje supervisado. Este conjunto de datos representa un reto significativo debido a la gran cantidad de clases y la similitud visual entre algunas razas, lo que lo convierte en un recurso ideal para evaluar la capacidad de generalización de los modelos de aprendizaje profundo.



En este proyecto, no fue necesaria ninguna limpieza de datos. El dataset se encuentra en un formato estructurado y completo, sin valores nulos ni inconsistencias. Sin embargo, durante el preprocesamiento, se aplicó una estandarización de los datos mediante la normalización de los valores de píxeles al rango [0,1], lo que facilita la convergencia del modelo durante el entrenamiento. Esta normalización se realizó dividiendo cada valor de píxel entre 255, el máximo valor posible en una imagen en escala RGB.



In [3]:

#cantidad de clases y ejemplos por clase
class_names = info.features['label'].names
num_classes = len(class_names)
print(f"Total de clases: {num_classes}")
print(f"Clases: {class_names}")

print(f"Ejemplos de entrenamiento: {len(list(raw_train))}")
print(f"Ejemplos de prueba: {len(list(raw_test))}")

# Verificar si hay clases desbalanceadas
class_counts = np.zeros(num_classes)
for _, label in raw_train:
    class_counts[label.numpy()] += 1

print(f"Distribución de clases en el entrenamiento: {class_counts}")

Total de clases: 120
Clases: ['n02085620-chihuahua', 'n02085782-japanese_spaniel', 'n02085936-maltese_dog', 'n02086079-pekinese', 'n02086240-shih-tzu', 'n02086646-blenheim_spaniel', 'n02086910-papillon', 'n02087046-toy_terrier', 'n02087394-rhodesian_ridgeback', 'n02088094-afghan_hound', 'n02088238-basset', 'n02088364-beagle', 'n02088466-bloodhound', 'n02088632-bluetick', 'n02089078-black-and-tan_coonhound', 'n02089867-walker_hound', 'n02089973-english_foxhound', 'n02090379-redbone', 'n02090622-borzoi', 'n02090721-irish_wolfhound', 'n02091032-italian_greyhound', 'n02091134-whippet', 'n02091244-ibizan_hound', 'n02091467-norwegian_elkhound', 'n02091635-otterhound', 'n02091831-saluki', 'n02092002-scottish_deerhound', 'n02092339-weimaraner', 'n02093256-staffordshire_bullterrier', 'n02093428-american_staffordshire_terrier', 'n02093647-bedlington_terrier', 'n02093754-border_terrier', 'n02093859-kerry_blue_terrier', 'n02093991-irish_terrier', 'n02094114-norfolk_terrier', 'n02094258-norwich_ter

El dataset está dividido en dos subconjuntos: 12,000 imágenes para el entrenamiento y 8,580 imágenes para la prueba. Esta división sigue las mejores prácticas en aprendizaje profundo, permitiendo medir de manera efectiva la capacidad del modelo para generalizar en datos no vistos. Las imágenes se almacenan en formato TFRecord, un formato binario eficiente para el manejo de grandes volúmenes de datos durante el entrenamiento.

### Desarrollo del Modelo de Deep Learning

El primer paso consiste en preparar los datos mediante un preprocesamiento adecuado. Las imágenes se redimensionan al tamaño 224x224 para coincidir con el formato requerido por MobileNetV2, y los valores de píxeles se escalan al rango [0,1] mediante una simple normalización, dividiendo cada valor entre 255. Este preprocesamiento se aplica tanto al conjunto de entrenamiento como al de prueba mediante la función map() de TensorFlow, asegurando que los datos sean cargados y preparados de forma eficiente utilizando prefetch() para mejorar el rendimiento durante el entrenamiento.

La estructura del modelo comienza definiendo una capa de entrada con una forma (224, 224, 3), la cual corresponde a imágenes a color en formato RGB. Posteriormente, se carga MobileNetV2 como el modelo base, excluyendo la capa superior (include_top=False) para permitir la adición de capas personalizadas específicas para este proyecto. Las capas internas de MobileNetV2 se congelan (base_model.trainable = False) para evitar que los pesos preentrenados se modifiquen durante la primera fase del entrenamiento, preservando así los patrones generales aprendidos de ImageNet.

Sobre esta base se añaden nuevas capas personalizadas que incluyen una capa de GlobalAveragePooling2D para reducir la dimensionalidad** de las características extraídas por MobileNetV2, seguida de una capa de Dropout con una probabilidad de 0.2 para prevenir el sobreajuste. Finalmente, se añade una capa completamente conectada (`Dense`) con 120 neuronas y una activación softmax para realizar la clasificación entre las 120 razas del dataset Stanford Dogs.

El modelo se construye utilizando la API funcional de Keras, donde las entradas del modelo base son conectadas con las capas superiores añadidas. Posteriormente, el modelo se compila utilizando el optimizador Adam y la función de pérdida sparse categorical crossentropy, la cual es adecuada para tareas de clasificación multiclase con etiquetas enteras. Como métrica de evaluación, se utiliza la precisión (accuracy) para medir el rendimiento del modelo en el conjunto de datos.

Se realiza una inspección de la arquitectura del modelo mediante la función summary(), la cual muestra que el modelo tiene un total de 2,411,704 parámetros, de los cuales 153,720 son entrenables y 2,257,984 no entrenables. Esto significa que únicamente las capas superiores añadidas serán ajustadas durante el entrenamiento inicial, mientras que los pesos del modelo base permanecerán fijos. Esta estrategia de Transfer Learning permite reducir el riesgo de sobreajuste y acelerar el proceso de convergencia del modelo, todo esto mencionado se puede observar en los siguientes bloques de código.



In [4]:
IMG_SIZE = (224, 224)  # Tamaño estándar

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE) / 255.0
    return image, label

# Preprocesamiento
train = raw_train.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
test = raw_test.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

In [5]:

input_tensor = Input(shape=(224, 224, 3))  # RGB 224x224

base_model = MobileNetV2(input_tensor=input_tensor, include_top=False, weights='imagenet')
base_model.trainable = False  

# Añadir capas superiores
x = base_model.output
x = GlobalAveragePooling2D()(x)  
x = Dropout(0.2)(x)  # Evitar overfitting
output = Dense(120, activation='softmax')(x)  

# modelo final
model = Model(inputs=base_model.input, outputs=output)

# modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# arquitectura del modelo
model.summary()



C:\Users\Alan\AppData\Local\Temp\ipykernel_15368\244740858.py:9: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_tensor=input_tensor, include_top=False, weights='imagenet')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,411,704 (9.20 MB)

 Trainable params: 153,720 (600.47 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [6]:
# Entrenar el modelo
EPOCHS = 10 
history = model.fit(
    train,
    validation_data=test,
    epochs=EPOCHS
)

Epoch 1/10


c:\Users\Alan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor']. Received: the structure of inputs=*
  warnings.warn(


375/375 ━━━━━━━━━━━━━━━━━━━━ 515s 1s/step - accuracy: 0.4279 - loss: 2.5305 - val_accuracy: 0.7449 - val_loss: 0.8603
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 484s 1s/step - accuracy: 0.8124 - loss: 0.6253 - val_accuracy: 0.7671 - val_loss: 0.7674
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 435s 1s/step - accuracy: 0.8657 - loss: 0.4442 - val_accuracy: 0.7709 - val_loss: 0.7551
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 349s 930ms/step - accuracy: 0.9010 - loss: 0.3414 - val_accuracy: 0.7718 - val_loss: 0.7556
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 367s 980ms/step - accuracy: 0.9256 - loss: 0.2702 - val_accuracy: 0.7761 - val_loss: 0.7506
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 349s 932ms/step - accuracy: 0.9450 - loss: 0.2171 - val_accuracy: 0.7693 - val_loss: 0.7630
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 339s 903ms/step - accuracy: 0.9608 - loss: 0.1757 - val_accuracy: 0.7749 - val_loss: 0.7646
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 341s 908ms/step - accuracy: 0.9641 - loss: 0.1547 - val_

El modelo fue entrenado durante 10 épocas, obteniendo un desempeño sólido tanto en los datos de entrenamiento como en los de validación. A lo largo del entrenamiento, el accuracy en el conjunto de validación alcanzó un 77.75*, mientras que el modelo logró una precisión del 97.67% en los datos de entrenamiento. Estos resultados reflejan la capacidad del modelo para aprender patrones significativos de las imágenes de perros y generalizar bien en datos no vistos, lo que sugiere que el uso de MobileNetV2 con Transfer Learning fue efectivo para esta tarea de clasificación.

In [7]:
# Evaluar el modelo en los datos de prueba
test_loss, test_accuracy = model.evaluate(test)
print(f"\nTest Accuracy: {test_accuracy:.4f}")


269/269 ━━━━━━━━━━━━━━━━━━━━ 159s 588ms/step - accuracy: 0.7754 - loss: 0.8013

Test Accuracy: 0.7751


In [8]:
# Imprimir el accuracy de cada época de forma estructurada
for epoch in range(EPOCHS):
    train_acc = history.history['accuracy'][epoch]
    val_acc = history.history['val_accuracy'][epoch]
    print(f"Epoch {epoch + 1}: Training Accuracy = {train_acc:.4f}, Validation Accuracy = {val_acc:.4f}")


Epoch 1: Training Accuracy = 0.5934, Validation Accuracy = 0.7449
Epoch 2: Training Accuracy = 0.8092, Validation Accuracy = 0.7671
Epoch 3: Training Accuracy = 0.8681, Validation Accuracy = 0.7709
Epoch 4: Training Accuracy = 0.9022, Validation Accuracy = 0.7718
Epoch 5: Training Accuracy = 0.9301, Validation Accuracy = 0.7761
Epoch 6: Training Accuracy = 0.9465, Validation Accuracy = 0.7693
Epoch 7: Training Accuracy = 0.9633, Validation Accuracy = 0.7749
Epoch 8: Training Accuracy = 0.9675, Validation Accuracy = 0.7756
Epoch 9: Training Accuracy = 0.9707, Validation Accuracy = 0.7784
Epoch 10: Training Accuracy = 0.9768, Validation Accuracy = 0.7751


In [11]:

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

fig = make_subplots()

fig.add_trace(go.Scatter(
    y=accuracy,
    mode='lines+markers',
    name='Training Accuracy'
))

fig.add_trace(go.Scatter(
    y=val_accuracy,
    mode='lines+markers',
    name='Validation Accuracy'
))

#  gráfico
fig.update_layout(
    title='Training and Validation Accuracy',
    xaxis=dict(title='Epoch'),
    yaxis=dict(title='Accuracy'),
    legend=dict(x=0.02, y=0.98),
)

fig.show()



### Resultados e Interpretación 1

Observamos que el accuracy de entrenamiento crece de manera consistente, alcanzando casi un 98% al final de las 10 épocas. Esto indica que el modelo ha aprendido patrones específicos de las imágenes del dataset, mostrando una alta precisión en los datos con los que fue entrenado.

Por otro lado, el accuracy de validación se estabiliza en torno al 77.5% después de las primeras tres épocas. Este comportamiento sugiere que el modelo ha aprendido lo suficiente para generalizar en los datos de prueba, aunque la diferencia entre el accuracy de entrenamiento y validación indica la posibilidad de un ligero sobreajuste. Sin embargo, la curva de validación no muestra una caída significativa, lo que implica que el modelo mantiene una buena capacidad de generalización.

### Ajuste de Hiperparámetros para evitar Overfitting

Durante el proceso de ajuste de hiperparámetros para evitar el overfitting, se implementaron varias técnicas clave. En primer lugar, se aplicó Data Augmentation mediante transformaciones aleatorias como volteo horizontal, ajuste de brillo y zoom aleatorio. Estas técnicas incrementaron la variabilidad del dataset y ayudaron a mejorar la capacidad de generalización del modelo. Adicionalmente, se aplicó Dropout con una tasa del 0.2 en las capas superiores, lo que redujo la posibilidad de que el modelo se sobreajustara a los datos de entrenamiento.

Luego del entrenamiento inicial, se habilitó el Fine-Tuning al descongelar las capas del modelo base MobileNetV2 para permitir que se ajustaran ligeramente a la tarea específica de clasificación de razas de perros. Este ajuste se realizó utilizando un optimizador Adam con un learning rate reducido a 1e-5, preservando así los patrones aprendidos y evitando grandes cambios que pudieran deteriorar el rendimiento del modelo.

El entrenamiento se llevó a cabo durante 10 épocas adicionales, y los resultados finales mostraron un accuracy de validación del 72.64%, mejorando significativamente con respecto a los intentos iniciales. Además, los pesos del modelo se guardaron después de cada ajuste importante, asegurando que las mejores versiones del modelo pudieran ser reutilizadas o evaluadas posteriormente sin necesidad de repetir todo el proceso de entrenamiento desde cero.


In [27]:

def random_zoom(image, min_zoom=0.8, max_zoom=1.0):
    img_shape = tf.shape(image)[:2]

    # zoom aleatorio 
    zoom_factor = tf.random.uniform((), min_zoom, max_zoom)

    new_size = tf.cast(zoom_factor * tf.cast(img_shape, tf.float32), tf.int32)

    image = tf.image.resize(image, new_size)

    image = tf.image.resize_with_crop_or_pad(image, img_shape[0], img_shape[1])
    return image

# Data Augmentation corregido
def augment(image, label):
    image = tf.image.random_flip_left_right(image)  
    image = tf.image.random_brightness(image, max_delta=0.1)  
    image = random_zoom(image, min_zoom=0.8, max_zoom=1.0)  
    return image, label

# Preprocesamiento de imágenes
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0 
    return image, label

#  datasets
train_dataset = raw_train.map(preprocess).map(augment).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = raw_test.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)


In [28]:
base_model.trainable = True  

# Compilar nuevamente 
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-Tuning del modelo
history_fine = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=10,  
    callbacks=[early_stopping]
)

# Evaluación final 
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Fine-Tuning - Test Loss: {test_loss:.4f}")
print(f"Fine-Tuning - Test Accuracy: {test_accuracy:.4f}")


Epoch 1/10


c:\Users\Alan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\models\functional.py:225: UserWarning:

The structure of `inputs` doesn't match the expected structure: ['keras_tensor_314']. Received: the structure of inputs=*



375/375 ━━━━━━━━━━━━━━━━━━━━ 1755s 4s/step - accuracy: 0.0127 - loss: 5.1900 - val_accuracy: 0.0606 - val_loss: 4.4164
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 1360s 4s/step - accuracy: 0.0517 - loss: 4.4864 - val_accuracy: 0.2203 - val_loss: 3.7308
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 1611s 4s/step - accuracy: 0.1453 - loss: 3.9130 - val_accuracy: 0.3911 - val_loss: 3.0021
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 1515s 4s/step - accuracy: 0.2649 - loss: 3.2728 - val_accuracy: 0.5146 - val_loss: 2.3884
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 1157s 3s/step - accuracy: 0.3797 - loss: 2.6823 - val_accuracy: 0.5966 - val_loss: 1.9399
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 977s 3s/step - accuracy: 0.4444 - loss: 2.2706 - val_accuracy: 0.6471 - val_loss: 1.6398
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 962s 3s/step - accuracy: 0.4970 - loss: 2.0022 - val_accuracy: 0.6749 - val_loss: 1.4383
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 1010s 3s/step - accuracy: 0.5344 - loss: 1.7825 - val_accuracy:

In [30]:
# Guardar los pesos 
model.save_weights('modelo_pesos.weights.h5')

# Cargar los pesos 
model.load_weights('modelo_pesos.weights.h5')

# entrenamiento 
history_fine_continue = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=10,  # Nuevas épocas adicionales
    callbacks=[early_stopping]
)

test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Continued Fine-Tuning - Test Loss: {test_loss:.4f}")
print(f"Continued Fine-Tuning - Test Accuracy: {test_accuracy:.4f}")


Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 931s 2s/step - accuracy: 0.6234 - loss: 1.3660 - val_accuracy: 0.7297 - val_loss: 1.0502
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 896s 2s/step - accuracy: 0.6434 - loss: 1.2904 - val_accuracy: 0.7354 - val_loss: 0.9924
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 851s 2s/step - accuracy: 0.6582 - loss: 1.2142 - val_accuracy: 0.7411 - val_loss: 0.9529
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 842s 2s/step - accuracy: 0.6780 - loss: 1.1280 - val_accuracy: 0.7484 - val_loss: 0.9124
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 844s 2s/step - accuracy: 0.6884 - loss: 1.0814 - val_accuracy: 0.7505 - val_loss: 0.8892
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 841s 2s/step - accuracy: 0.7109 - loss: 1.0030 - val_accuracy: 0.7545 - val_loss: 0.8604
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 846s 2s/step - accuracy: 0.7247 - loss: 0.9491 - val_accuracy: 0.7608 - val_loss: 0.8374
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 842s 2s/step - accuracy: 0.7300 - loss: 0.9022 - val_accu

In [31]:
model.save_weights('pesos_fine_tuning.weights.h5')

model.load_weights('pesos_fine_tuning.weights.h5')

history_fine_continue = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=5,  
    callbacks=[early_stopping]
)


Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 1011s 3s/step - accuracy: 0.7642 - loss: 0.8103 - val_accuracy: 0.7659 - val_loss: 0.7841
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 955s 3s/step - accuracy: 0.7679 - loss: 0.7867 - val_accuracy: 0.7693 - val_loss: 0.7716
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 876s 2s/step - accuracy: 0.7817 - loss: 0.7323 - val_accuracy: 0.7717 - val_loss: 0.7643
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 863s 2s/step - accuracy: 0.7951 - loss: 0.6996 - val_accuracy: 0.7721 - val_loss: 0.7564
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 862s 2s/step - accuracy: 0.8096 - loss: 0.6421 - val_accuracy: 0.7739 - val_loss: 0.7486


Para mejorar el accuracy del modelo, se decidió realizar entrenamientos adicionales mediante la técnica de Fine-Tuning. En cada etapa del entrenamiento, los pesos anteriores se guardaron para asegurar que las mejores configuraciones alcanzadas se preservaran y pudieran ser reutilizadas. Esto permitió retomar el entrenamiento desde el punto donde se habían obtenido los mejores resultados, evitando así la necesidad de comenzar de nuevo cada vez.

Durante esta fase, se cargaron los pesos guardados y se agregaron más épocas de entrenamiento con el objetivo de continuar ajustando el modelo a las características del dataset. En la segunda etapa de Fine-Tuning, se entrenó el modelo por cinco épocas adicionales, alcanzando un accuracy en validación del 77.39% y logrando así una mejora con respecto a los intentos previos. Este enfoque permitió que el modelo refinara su capacidad de generalización, ajustando mejor los patrones aprendidos sin perder los progresos anteriores. 

El uso de Early Stopping garantizó que el entrenamiento no continuara innecesariamente si no se observaban mejoras significativas, evitando así el riesgo de sobreajuste y optimizando los recursos computacionales.

In [37]:

# Gráfica
fig = make_subplots()

fig.add_trace(go.Scatter(
    y=history_fine_continue.history['accuracy'],
    mode='lines+markers',
    name='Training Accuracy'
))

fig.add_trace(go.Scatter(
    y=history_fine_continue.history['val_accuracy'],
    mode='lines+markers',
    name='Validation Accuracy'
))

fig.update_layout(
    title='Training and Validation Accuracy after Fine-Tuning',
    xaxis=dict(title='Epoch'),
    yaxis=dict(title='Accuracy')
)

fig.show()


La gráfica refleja el desempeño del modelo durante el Fine-Tuning, mostrando la evolución del accuracy tanto en el conjunto de entrenamiento como en el de validación a lo largo de las cinco épocas adicionales. 

Se observa que el accuracy de entrenamiento mejora de manera consistente, alcanzando valores superiores al 81% en la última época. Esto indica que el modelo continúa ajustando sus parámetros de manera efectiva, aprendiendo patrones relevantes del dataset sin signos de estancamiento.

Por otro lado, el accuracy de validación también muestra una ligera mejora, comenzando cerca del 77.5% y alcanzando un 77.7% al final del proceso. Aunque el incremento no es tan pronunciado como en el conjunto de entrenamiento, refleja que el modelo mantiene su capacidad de generalización en datos no vistos. La diferencia entre ambas curvas sugiere que el modelo ha seguido aprendiendo, pero sin sobreajustarse significativamente a los datos de entrenamiento.



In [33]:

y_pred = np.argmax(model.predict(test_dataset), axis=-1)
y_true = np.concatenate([y for x, y in test_dataset], axis=0)

# Crear matriz de confusión
cm = confusion_matrix(y_true, y_pred)

fig = px.imshow(
    cm, 
    text_auto=True, 
    color_continuous_scale='Blues',
    labels=dict(x="Predicted Label", y="True Label", color="Count"),
    title='Confusion Matrix'
)

fig.update_layout(
    xaxis=dict(tickmode='linear'),
    yaxis=dict(tickmode='linear'),
    width=800, 
    height=600
)

fig.show()


c:\Users\Alan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\models\functional.py:225: UserWarning:

The structure of `inputs` doesn't match the expected structure: ['keras_tensor_314']. Received: the structure of inputs=*



269/269 ━━━━━━━━━━━━━━━━━━━━ 101s 368ms/step


La matriz de confusión refleja el rendimiento del modelo en términos de las predicciones realizadas para cada clase del dataset. En el eje vertical se encuentran las etiquetas verdaderas (True Label), mientras que en el eje horizontal se muestran las etiquetas predichas (Predicted Label). Cada celda representa el número de instancias para las cuales el modelo realizó una predicción específica.

En esta matriz se observa que, aunque la mayoría de las predicciones se agrupan cerca de la diagonal (lo que indica que el modelo acertó al predecir la clase correcta), también hay una dispersión considerable de puntos fuera de la diagonal. Esto sugiere que el modelo tiene dificultades para diferenciar entre algunas clases de razas de perros, probablemente debido a las similitudes visuales entre algunas de ellas.

Además, los valores más oscuros a lo largo de la diagonal indican aquellas clases donde el modelo tuvo un mejor rendimiento, mientras que los colores más claros sugieren clases donde el desempeño fue menos preciso. La dispersión en otras partes de la matriz refleja la confusión del modelo al asignar instancias a clases incorrectas.


### Conclusión


En este trabajo se desarrolló un modelo de Deep Learning utilizando Transfer Learning con la arquitectura MobileNetV2, para abordar la clasificación de imágenes del dataset Stanford Dogs, que contiene 120 clases de razas de perros. A lo largo del proceso, se aplicaron diversas técnicas como preprocesamiento de imágenes, Data Augmentation, Fine-Tuning, y el uso de Early Stopping, con el objetivo de optimizar el rendimiento del modelo y evitar el sobreajuste.

El entrenamiento inicial del modelo mostró un buen desempeño, con un accuracy de validación cercano al 77.5%. Sin embargo, se detectaron áreas de mejora al analizar las curvas de aprendizaje y la matriz de confusión, lo que nos llevó a implementar ajustes adicionales mediante Fine-Tuning y la reutilización de pesos previos para continuar el entrenamiento. Gracias a estas técnicas, se logró mejorar progresivamente el modelo, alcanzando un accuracy final del 77.39% tras cinco épocas adicionales. 

La matriz de confusión evidenció que, aunque el modelo aprendió a diferenciar muchas clases correctamente, aún existe cierta confusión entre algunas razas debido a su alta similitud visual. Esto sugiere que, en futuros trabajos, podrían explorarse técnicas más avanzadas de regularización o arquitecturas más complejas para mejorar aún más la capacidad del modelo de distinguir entre clases difíciles.

